<a href="https://colab.research.google.com/github/eftekhar-hossain/Disaster_IEEE-Access/blob/main/damage_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [16]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re,nltk,json
from sklearn.metrics import confusion_matrix,classification_report 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from keras.utils.vis_utils import plot_model
np.random.seed(42)
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import string, spacy,unicodedata, random
from bs4 import BeautifulSoup
class color: # Text style
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
import warnings
warnings.filterwarnings('ignore')
### Pretrained Word Embeddings
pretrained_path = "/Users/zaima/Desktop/Dataset/PreTrained Wordembedding/"
# dataset_path = "/content/drive/MyDrive/Colab Notebooks/MSC Presentation Tasks/Disaster Response/Images/"
# folder_path = "/content/drive/MyDrive/Colab Notebooks/MSC Presentation Tasks/Disaster Response/"
models_path = "/Users/zaima/Desktop/Dataset/Multimodal Sentiment/"


folder_path = "/Users/zaima/Desktop/Dataset/"
# models_path = "/content/drive/MyDrive/TaheriThesis/Dataset/Multimodal Sentiment/Models/"
# results_path = "/content/drive/MyDrive/TaheriThesis/Dataset/Multimodal Sentiment/Results/"
dataset_path = "/Users/zaima/Desktop/Dataset/Images/"

CPU times: user 1.65 ms, sys: 807 µs, total: 2.45 ms
Wall time: 1.8 ms


[nltk_data] Downloading package punkt to /Users/zaima/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import ResNet152
from keras.applications.resnet_v2 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, GlobalAveragePooling2D, Embedding, LSTM, multiply
from keras.models import Model
from keras import preprocessing, Input
from tensorflow import keras
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
# from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from tensorflow.keras.models import load_model
import itertools
from PIL import Image, ImageFile
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D,MaxPooling1D,Conv1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.layers import Add, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import RMSprop,Adam,SGD,Nadam
keras.backend.clear_session()

#Fetching Dataset

In [18]:
# train_data = pd.read_excel(folder_path+'train.xlsx')
# test_data = pd.read_csv(folder_path+'test.xlsx')
# valid_data = pd.read_excel(main_path+'valid_new.xlsx')



train_data = pd.read_csv(folder_path+'train.csv')
valid_data = pd.read_csv(folder_path+'val.csv')
test_data = pd.read_csv(folder_path+'test.csv')



In [19]:
train_data.columns

Index(['image_name', 'Captions', 'Label_Sentiment', 'Label'], dtype='object')

In [20]:
train_data['Label'].value_counts()

Label
0    861
6    716
4    623
1    477
5    408
3    400
2    348
Name: count, dtype: int64

In [21]:
test_data['Label'].value_counts()

Label
0    93
6    79
4    68
5    46
3    44
1    44
2    40
Name: count, dtype: int64

In [22]:



train_data['enc_label'] = train_data['Label_Sentiment'].replace({'happy':0,'angry':1,'disgust':2,'fear':3,'sad':4,'surprise':5,'other':6})
valid_data['enc_label'] = valid_data['Label_Sentiment'].replace({'happy':0,'angry':1,'disgust':2,'fear':3,'sad':4,'surprise':5,'other':6})
test_data['enc_label'] = test_data['Label_Sentiment'].replace({'happy':0,'angry':1,'disgust':2,'fear':3,'sad':4,'surprise':5,'other':6})

In [23]:
print("Size of the training data ==> ", train_data.shape)
print("Size of the Test data ==>", test_data.shape)

Size of the training data ==>  (3833, 5)
Size of the Test data ==> (414, 5)


## Image Fetching

In [24]:
## collect image names from the folders
def create_img_array(img_dirct):
    all_imgs = []
    for root, j, files in os.walk(img_dirct):
        for file in files:
            file = root + '' + file
            all_imgs.append(file)
    return all_imgs

def create_img_path(DF, Col_name, img_dir):
    img_path = [img_dir + '' + name for name in DF[Col_name]]
    return img_path

In [25]:
# Creating train, test and validation image path
train_img_path = create_img_path(train_data,'image_name', dataset_path)
valid_img_path = create_img_path(valid_data,'image_name', dataset_path)

In [26]:
test_img_path = create_img_path(test_data,'image_name', dataset_path)

In [27]:
# Function that returns image reading from the path
def get_input(path):
    # Loading image from given path
    # and resizing it to 150*150*3 format
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    img= tf.keras.utils.load_img(path, target_size=(224,224)) 
    # img.close()   
    return(img)

# Takes in image and preprocess it
def process_input(img):
    # Converting image to array    
    img_data =tf.keras.utils.img_to_array(img)
    # Adding one more dimension to array    
    img_data = np.expand_dims(img_data, axis=0)
    #     
    img_data = preprocess_input(img_data)
    #img_data = preprocess_input(img_data)
    return(img_data)


In [28]:
# Create an array of training images
train_images = []
for n,i in enumerate(train_img_path):
  input_img = get_input(i)
  process_img = process_input(input_img)
  # print(n)
  train_images.append(process_img[0])

In [29]:
# Create an array of training images
valid_images = []
for n,i in enumerate(valid_img_path):
  input_img = get_input(i)
  process_img = process_input(input_img)
  # print(n)
  valid_images.append(process_img[0])

In [30]:
# Create an array of training images
test_images = []
for n,i in enumerate(test_img_path):
  input_img = get_input(i)
  process_img = process_input(input_img)
  # print(n)
  test_images.append(process_img[0])


In [31]:
# convert into numpy array
train_image = np.array(train_images)

In [32]:
valid_image = np.array(valid_images)

In [33]:
test_image = np.array(test_images)

In [34]:
import pickle as pkl
with open(dataset_path+'train.pkl','wb') as f:
    pkl.dump(train_image, f)

# with open(dataset_path+'test.pkl','wb') as f:
#     pkl.dump(test_image, f)    

In [35]:
import pickle as pkl
with open(dataset_path+'valid.pkl','wb') as f:
    pkl.dump(valid_image, f)

In [36]:
import pickle as pkl


with open(dataset_path+'test.pkl','wb') as f:
    pkl.dump(test_image, f)  

In [37]:

with open(dataset_path+'train.pkl','rb') as f:
  train_image = pkl.load(f)
  print("Training Images:-- ",train_image.shape)

Training Images:--  (3833, 224, 224, 3)


In [38]:

with open(dataset_path+'valid.pkl','rb') as f:
  valid_image = pkl.load(f)
  print("Validation Images:-- ",valid_image.shape)

Validation Images:--  (414, 224, 224, 3)


In [39]:
with open(dataset_path+'test.pkl','rb') as f:
  test_image = pkl.load(f)
  print("Test Images:-- ",test_image.shape)

Test Images:--  (414, 224, 224, 3)


## One Hot Encoding

In [40]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [41]:
# encoder=OneHotEncoder(sparse=False)

# encoded_labels = pd.DataFrame (encoder.fit_transform(train_data[['label']]))

# encoded_labels .columns = encoder.get_feature_names(['label'])

# train_data= pd.concat([train_data, encoded_labels ], axis=1)

import pandas as pd

# Define the list of categories for your Label_Sentiment variable
categories = ['happy', 'angry', 'disgust', 'fear', 'sad', 'surprise', 'other']

# Perform one-hot encoding using get_dummies() function
encoded_labels = pd.get_dummies(train_data['Label_Sentiment'], columns=categories, prefix='Label_Sentiment')

# Concatenate the encoded labels with your original train_data DataFrame
train_data = pd.concat([train_data, encoded_labels], axis=1)

In [42]:
train_data.columns

Index(['image_name', 'Captions', 'Label_Sentiment', 'Label', 'enc_label',
       'Label_Sentiment_angry', 'Label_Sentiment_disgust',
       'Label_Sentiment_fear', 'Label_Sentiment_happy',
       'Label_Sentiment_other', 'Label_Sentiment_sad',
       'Label_Sentiment_surprise'],
      dtype='object')

In [43]:
# train_data = train_data.rename(columns={'label_damaged_infrastructure':'damaged_infrastructure', 'label_damaged_nature':'damaged_nature', 
#                                         'label_fires':'fires','label_flood':'flood',
#                                         'label_human_damage':'human_damage', 'label_non_damage':'non_damage'})


train_data = train_data.rename(columns={'Label_Sentiment_happy':'happy', 
                                        'Label_Sentiment_angry':'angry',
                                        'Label_Sentiment_disgust':'disgust',
                                        'Label_Sentiment_fear':'fear',
                                        'Label_Sentiment_sad':'sad',
                                        'Label_Sentiment_surprise':'surprise',
                                        'Label_Sentiment_other':'other'})
     

train_data

,image_name,Captions,Label_Sentiment,Label,enc_label,angry,disgust,fear,happy,other,sad,surprise
0,205.jpg,নির্বোধ দেখার এবং আপনি যে যত্নশীল তা না করার ক...,other,6,6,False,False,False,False,True,False,False
1,209.jpg,একটি বোর্ডার আর্ম স্কেটার একটি আমন্ত্রণ উপার্জ...,other,6,6,False,False,False,False,True,False,False
2,210.jpg,যদি শরীর দুর্বল হয় তবে মন শক্তিশালী হবে না।,other,6,6,False,False,False,False,True,False,False
3,215.jpg,একজন ধার্মিক ব্যক্তির কার্যকর উত্সাহ প্রার্থনা...,other,6,6,False,False,False,False,True,False,False
4,222.jpg,অসমাপ্ত ?,other,6,6,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
3828,4676.jpg,অভিনন্দন ২ প্রথম আইবিএম প্যাথওয়েসটেক পণ্ডিতরা...,happy,0,0,False,False,False,True,False,False,False
3829,4677.jpg,সবেমাত্র আমার চুলের সাথে রুটোশায়ার স্ক্যাল্প ...,happy,0,0,False,False,False,True,False,False,False
3830,4678.jpg,সবেমাত্র এই ওক ড্রেসারটি শেষ করেছেন।,happy,0,0,False,False,False,True,False,False,False
3831,4682.jpg,সুগারম্যান রজার্স মামলা-মোকদ্দমা কেমব্রিজে গত ...,happy,0,0,False,False,False,True,False,False,False


In [44]:
train_data.columns

Index(['image_name', 'Captions', 'Label_Sentiment', 'Label', 'enc_label',
       'angry', 'disgust', 'fear', 'happy', 'other', 'sad', 'surprise'],
      dtype='object')

In [45]:
train_data.Captions = train_data.Captions.astype(str)

## Text Cleaning

In [46]:

import locale
print(locale.getpreferredencoding())

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

UTF-8


In [47]:
!pip install emoji

In [48]:
'''
Text Cleaning
'''
# import emoji
# import re

# def remove_emojis(text):
#     emoji_pattern = re.compile("["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            "]+", flags=re.UNICODE)
#     return emoji_pattern.sub(r'', text)


def text_cleaning(row):
   #to remove HTML tags
  text = BeautifulSoup(row, 'html.parser').get_text()
  d = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) #This line is for removing url
  post = d.replace('\n', '')
  post = d.replace('\r\n', '')
  post = post.replace('—', ' ') 
  post = post.replace('।', ' ')
  text = ''.join([c for c in post if c not in string.punctuation])
#   text= remove_emojis(text)
  # to remove special characters
  pattern = r'^\s*|\s\s*'
  text = re.sub(pattern, ' ', text).strip()
  # convert into lower case
  text = text.lower() 
  # # Stopword 
  # result = text.split()
  # text = [word.strip() for word in result if word not in stp ]
  # text =" ".join(text)
  
  return text

In [49]:
#Removing punctuations
train_data['cleaned'] = train_data['Captions'].apply(text_cleaning)
valid_data['cleaned'] = valid_data['Captions'].apply(text_cleaning)
test_data['cleaned'] = test_data['Captions'].apply(text_cleaning)


## Data samples after cleaning
print("Data samples after cleaning:\n")
for i in range(10):
  if i in train_data.index:
    print("Original Data:===\n",train_data.Captions[i],"\nCleaned Data:===\n",train_data.cleaned[i],)

Data samples after cleaning:

Original Data:===
 নির্বোধ দেখার এবং আপনি যে যত্নশীল তা না করার ক্ষমতা আছে 
Cleaned Data:===
 নির্বোধ দেখার এবং আপনি যে যত্নশীল তা না করার ক্ষমতা আছে
Original Data:===
 একটি বোর্ডার আর্ম স্কেটার একটি আমন্ত্রণ উপার্জন করবে? 
Cleaned Data:===
 একটি বোর্ডার আর্ম স্কেটার একটি আমন্ত্রণ উপার্জন করবে
Original Data:===
 যদি শরীর দুর্বল হয় তবে মন শক্তিশালী হবে না। 
Cleaned Data:===
 যদি শরীর দুর্বল হয় তবে মন শক্তিশালী হবে না
Original Data:===
 একজন ধার্মিক ব্যক্তির কার্যকর উত্সাহ প্রার্থনা অনেকটা উপকার পায়। 
Cleaned Data:===
 একজন ধার্মিক ব্যক্তির কার্যকর উত্সাহ প্রার্থনা অনেকটা উপকার পায়
Original Data:===
 অসমাপ্ত ? 
Cleaned Data:===
 অসমাপ্ত
Original Data:===
 আপনাকে পুরো হতে নিখুঁত হতে হবে না! 
Cleaned Data:===
 আপনাকে পুরো হতে নিখুঁত হতে হবে না
Original Data:===
 আজ আমাদের উঠোনে একটি ঠান্ডা, ক্লান্ত, ক্ষুধার্ত, আতঙ্কিত কালো বিড়াল পেয়েছে। সে সেখানে কয়েক দিন ছিল, কেউ তাকে মিস করছে? 
Cleaned Data:===
 আজ আমাদের উঠোনে একটি ঠান্ডা ক্লান্ত ক্ষুধার্ত আতঙ্কিত কা

##Evaluation Metrics

In [50]:
'''Evaluation Parameters'''

def print_metrices(true,pred):
    print(confusion_matrix(true,pred))
    print(classification_report(true,pred))
    print("Accuracy : ",accuracy_score(true,pred))
    print("Precison : ",precision_score(true,pred, average = 'weighted'))
    print("Recall : ",recall_score(true,pred,  average = 'weighted'))
    print("F1 : ",f1_score(true,pred,  average = 'weighted'))

## Confusion Matrix

In [51]:
## Confusion matrix function
def con_mat(true,pred,class_names,model_name):
  cm = confusion_matrix(true,pred)
  #sns.set()
  plt.figure(figsize =(4, 3))
  ax = plt.subplot()
 # 'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds','YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            #'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'
  sns.heatmap(cm, annot=True,fmt="d",cmap='YlOrRd', ax = ax,annot_kws={"size": 10},) #annot=True to annotate cells
  # labels, title and ticks
  ax.set_xlabel('Predicted labels',fontsize=10)
  ax.set_ylabel('True labels',fontsize=10) 
  #ax.set_title(f'Confusion Matrix of {model_name}',fontsize=10) 
  ax.xaxis.set_ticklabels(class_names, rotation=45); ax.yaxis.set_ticklabels(class_names, rotation=45);
  ax.xaxis.tick_top()
 # plt.savefig(folder_path + f"{model_name}.png",bbox_inches='tight',dpi =500)

In [52]:
from sklearn.utils import compute_class_weight
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_data['enc_label']),
                                        y = train_data['enc_label']                                                   
                                    )
class_weights = dict(zip(np.unique(train_data['enc_label']), class_weights))
class_weights

{0: 0.6359714617554338,
 1: 1.147948487571129,
 2: 1.5734811165845648,
 3: 1.3689285714285715,
 4: 0.881757533931447,
 5: 1.3355400696864113,
 6: 0.7647645650438947}

#Textual Models

## Tokenizer

In [53]:
''' Tokenizer '''

def text_tokenizer(train_data,valid_data,test_data,vocabulary,max_len,sample_text_num):
  
  tokenizer = Tokenizer(num_words = vocabulary ,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n-', 
                        split=' ', char_level=False, oov_token='<oov>', document_count=0)
  tokenizer.fit_on_texts(train_data['cleaned'])  
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1
  
  # Training Sequences
  train_sequences = tokenizer.texts_to_sequences(train_data['cleaned'])
  train_pad_sequences =  keras.preprocessing.sequence.pad_sequences(train_sequences, value=0.0, padding='post', maxlen= max_len)


  valid_sequences = tokenizer.texts_to_sequences(valid_data['cleaned'])
  valid_pad_sequences =  keras.preprocessing.sequence.pad_sequences(valid_sequences, value=0.0, padding='post', maxlen= max_len)
  
  # Test Sequences
  test_sequences = tokenizer.texts_to_sequences(test_data['cleaned'])
  test_pad_sequences =  keras.preprocessing.sequence.pad_sequences(test_sequences, value=0.0, padding='post', maxlen= max_len)
 

  print(color.BOLD+"\n\t\t\t====== Encoded Sequences ======"+color.END,"\n")  
  print(train_data.cleaned[sample_text_num],"\n",train_sequences[sample_text_num])
  print(color.BOLD+"\n\t\t\t====== Paded Sequences ======\n"+color.END,"\n",train_pad_sequences[sample_text_num]) 

  return train_pad_sequences, valid_pad_sequences, test_pad_sequences, vocab_size, word_index


vocabulary = 50000 
max_len = 180 
sample_text_num = 10

## Call Tokenizer
train_pad_sequences,valid_pad_sequences, test_pad_sequences, vocab_size, word_index =  text_tokenizer(train_data,valid_data,test_data,
                                                                      vocabulary,max_len,sample_text_num) 

print("Number of Train Sequences :" ,train_pad_sequences.shape)
print("Number of Train Sequences :" ,valid_pad_sequences.shape)
print("Number of Test Sequences :" ,test_pad_sequences.shape)
print("Vocabulary Size: ",vocab_size)


			====== Encoded Sequences ====== 

আজ সকালে একটি অন্ধকার লন্ডন কলনিতে আর্সেনালের প্রথম দলের ট্রেন আশা করি আমিরাতরা আগামীকাল হতাশ হবে না 
 [31, 279, 6, 380, 1117, 4152, 4153, 152, 805, 806, 206, 18, 4154, 531, 72, 38, 5]

			====== Paded Sequences ======
 
 [  31  279    6  380 1117 4152 4153  152  805  806  206   18 4154  531
   72   38    5    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0

In [54]:
## Convert labels into array
y_train = np.array(train_data['enc_label']).reshape(-1,1)
y_valid = np.array(valid_data['enc_label']).reshape(-1,1)
y_test = np.array(test_data['enc_label']).reshape(-1,1)

## Callbacks

In [55]:
''' Callbacks'''
keras.backend.clear_session()
def callbacks_check(model_name):
  num_classes = 7
  accuracy_threshold = 0.99

  class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>accuracy_threshold):
          print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
          self.model.stop_training = True

  acc_callback = myCallback()
  # Saved the Best Model
  filepath = models_path+'Models_3/'+f"{model_name}.h5"
  checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
  # callback list
  callback_list = [acc_callback, checkpoint] 

  return callback_list

## Textual Models

In [77]:

def create_keras_embeddings_models(max_len):

  ###### BiLSTM Model #######
  bi_text_inputs = Input(shape=(max_len,))
  bi_embedding_layer = Embedding(vocab_size,100)(bi_text_inputs)
  LSTM_Layer_1 = Bidirectional(LSTM(128,dropout=0.01))(bi_embedding_layer)
  bi_dense_layer_1 = Dense(7, activation='softmax')(LSTM_Layer_1)
  bilstm_model = Model(inputs=bi_text_inputs, outputs=bi_dense_layer_1)


  models = [bilstm_model]
 

  return models

In [78]:
from tensorflow.keras.utils import to_categorical
y_train=to_categorical(train_data['enc_label'])
y_valid=to_categorical(valid_data['enc_label'])
y_test=to_categorical(test_data['enc_label'])

In [79]:
y_train

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

#Visual Models

In [80]:

keras.backend.clear_session()


def visual_models():

  #### Resnet
  resnet = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  #resnet.trainable = False
  for layer in resnet.layers[0:-10]:
      layer.trainable = False
  # add a global spatial average pooling layer
  y = resnet.output
  pool1 = GlobalAveragePooling2D()(y)
  # let's add a fully-connected layer
  #flat1 = Flatten()(pool1)
  # and a logistic layer -- let's say we have 200 classes
  #hidden = Dense(512, activation='relu')(flat1)
  #dropout1 = Dropout(0.1)(hidden)
  output1 = Dense(7, activation='softmax')(pool1)
  # this is the model we will train
  resnet_img_model = Model(inputs=resnet.input, outputs=output1)



# base_model = 

# # Add a global average pooling layer to reduce spatial dimensions
# x = base_model.output
# x = GlobalAveragePooling2D()(x)

# # Add a fully connected layer with softmax activation for classification
# predictions = Dense(num_classes, activation='softmax')(x)

# # Create the model by specifying the input and output tensors
# model = Model(inputs=base_model.input, outputs=predictions)

  models = [resnet_img_model]

  return models


In [81]:
visual_model_name = ['resnet']
visual_models_lsit = visual_models()

# textual_model_name = ['LSTM','CNN','CNN_LSTM']
textual_model_name = ['BiLSTM']
text_models_list = create_keras_embeddings_models(180)


In [82]:
for vm,visual_model in enumerate(visual_models_lsit):
  for tm,textual_model in enumerate(text_models_list):
    # Concatenating the output of 2 classifiers
    con_layer = keras.layers.concatenate([visual_model.output, textual_model.output])
    dropout = Dropout(0.2)(con_layer)
    final_dense = Dense(64, activation="relu")(dropout)
    out = Dense(7,activation='softmax')(final_dense)
     
    #Defining model input and output
    com_model = Model(inputs = [visual_model.input, textual_model.input], outputs=out)
   
      
    com_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics = ["accuracy"])
    print(f"Model Name: {visual_model_name[vm]}+{textual_model_name[tm]}\n")
    com_model.fit([train_image,train_pad_sequences],
    y_train,
    epochs=50,
    batch_size =32,
    validation_data=([valid_image,valid_pad_sequences],y_valid),
    verbose =1,
    class_weight = class_weights,
    callbacks = callbacks_check(visual_model_name[vm]+textual_model_name[tm]))

Model Name: resnet+BiLSTM

Epoch 1/50


2023-07-07 01:36:01.144134: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


120/120 [==============================] - ETA: 0s - loss: 1.9447 - accuracy: 0.1492
Epoch 1: val_accuracy improved from -inf to 0.16425, saving model to /Users/zaima/Desktop/Dataset/Multimodal Sentiment/Models_3/resnetBiLSTM.h5
120/120 [==============================] - 105s 800ms/step - loss: 1.9447 - accuracy: 0.1492 - val_loss: 1.9272 - val_accuracy: 0.1643
Epoch 2/50
120/120 [==============================] - ETA: 0s - loss: 1.8915 - accuracy: 0.1936
Epoch 2: val_accuracy improved from 0.16425 to 0.19082, saving model to /Users/zaima/Desktop/Dataset/Multimodal Sentiment/Models_3/resnetBiLSTM.h5
120/120 [==============================] - 111s 929ms/step - loss: 1.8915 - accuracy: 0.1936 - val_loss: 1.9574 - val_accuracy: 0.1908
Epoch 3/50
120/120 [==============================] - ETA: 0s - loss: 1.8452 - accuracy: 0.2014
Epoch 3: val_accuracy improved from 0.19082 to 0.30918, saving model to /Users/zaima/Desktop/Dataset/Multimodal Sentiment/Models_3/resnetBiLSTM.h5
120/120 [======

## Multimodal Models Performance

In [83]:
multimodal_models = ['resnetBiLSTM']
multimodal_names =  ['ResNet+BiLSTM']

def multimodal_models_accuracy(saved_model): 
  my_dict = {}
  # Prediction 
  model = load_model(models_path+'Models_3/'+f"{saved_model}.h5")
  y_pred = np.argmax(model.predict([test_image,test_pad_sequences]), axis=-1)

  y_true = test_data['enc_label']

  my_dict['Accuracy'] = accuracy_score(y_true, y_pred)*100
  my_dict['Precision'] = precision_score(y_true, y_pred,average = 'weighted')*100
  my_dict['Recall'] = recall_score(y_true, y_pred,average = 'weighted')*100 
  my_dict['F1 Score'] = f1_score(y_true, y_pred,average = 'weighted')*100 
  return my_dict

In [84]:
accuracy = {f'{multimodal_names[i]}':multimodal_models_accuracy(model) for i,model in enumerate(multimodal_models)}
# Save the performance parameter into json file
with open(models_path+'Results_3/'+'anurag_models_performance.json', 'w') as f:
    json.dump(accuracy, f)

13/13 [==============================] - 17s 833ms/step


In [87]:
# Load the json file
metrics = json.load(open(models_path+'Results_3/'+'anurag_models_performance.json'))
acc_list = []
pr_list = []
re_list = []
f1_list = []
for i in metrics.keys():
  acc_list.append(round(metrics[i]['Accuracy'],2))
  pr_list.append(round(metrics[i]['Precision'],2))
  re_list.append(round(metrics[i]['Recall'],2))
  f1_list.append(round(metrics[i]['F1 Score'],2))

print (color.BOLD+f"=======  Multimodal Models Performance on Test Data  =============\n"+color.END)
# Create a dataframe
performance_matrix = pd.DataFrame({'Accuracy':acc_list,'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list},
                                  index =multimodal_names)
performance_matrix

=======  Multimodal Models Performance on Test Data  =============



,Accuracy,Precision,Recall,F1 Score
ResNet+BiLSTM,75.53,76.01,75.53,75.86
